# 1. 서로소 집합 자료구조

* 합집합(Union)
* 찾기(Find)

command: `Union(1,4)`, `Union(2,3)`, `Union(2,4)`, `Union(5,6)`

| 노드       | 1    | 2    | 3    | 4    | 5    | 6    |
| ---------- | ---- | ---- | ---- | ---- | ---- | ---- |
| 부모테이블 | 1    | 2    | 3    | 4    | 5    | 6    |
| U(1,4)     | 1    | 2    | 3    | 1    | 5    | 6    |
| U(2,3)     | 1    | 2    | 2    | 1    | 5    | 6    |
| U(2,4)     | 1    | 1    | 2    | 1    | 5    | 6    |
| U(5,6)     | 1    | 1    | 2    | 1    | 5    | 5    |

루트 노드를 보면 {1, 2, 3, 4}, {5, 6} 이 서로소 집합이다.

단점은 탐색을 하면 루트노드를 찾기 위해 부모테이블을 거슬러 올라가게 된다.

In [96]:
# 특정 원소가 속한 집합을 찾기
def find_parent(parent, x):
    # 루트 노드를 찾을 때까지 재귀 호출
    if parent[x] != x:
        return find_parent(parent, parent[x])
    return x

# 두 원소가 속한 집합을 합치기
def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b
        
# 노드의 개수와 간선(Union 연산)의 개수 입력 받기
v, e = map(int, input().split())
parent = [0] * (v + 1) # 부모 테이블 초기화하기

# 부모 테이블 상에서, 부모를 자기 자신으로 초기화
for i in range(1, v+1):
    parent[i] = i
    
# Union 연산을 각각 수행
for i in range(e):
    a, b = map(int, input().split())
    union_parent(parent, a, b)
    
# 각 원소가 속한 집합 출력하기
print('각 원소가 속한 집합: ', end='')
for i in range(1, v+1):
    print(find_parent(parent, i), end=' ')
print()

# 부모 테이블 출력하기
print('부모 테이블: ', end='')
for i in range(1, v+1):
    print(parent[i], end=' ')

6 4
1 4
2 3
2 4
5 6
각 원소가 속한 집합: 1 1 1 1 5 5 
부모 테이블: 1 1 1 1 5 5 

## 1.1 서로소 집합 자료구조의 단점

합집합 연산이 평향된 경우 find 함수의 시간 복잡도가 $O(V)$가 된다.

찾기 재귀함수를 최적화하기 위해 **경로 압축**을 사용한다.

> 그렇다고 parent list root node를 갖는 건 아니다.

In [95]:
# 특정 원소가 속한 집합을 찾기
def find_parent(parent, x):
    # 루트 노드를 찾을 때까지 재귀 호출
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

## 1.2 서로소 집합을 이용한 사이클 판별

무방향 그래프에서 사이클이 있는 지 없는 지 판별한다.

각 간선마다

    1. 루트 노드가 서로 다르다면 두 노드에 대해 합집합 연산을 한다.
    2. 루트 노드가 서로 같다면 사이클이 있는 것이다.

![](./fig/cycle.png)

| 노드       | 1    | 2    | 3    |
| ---------- | ---- | ---- | ---- |
| 부모테이블 | 1    | 2    | 3    |
| U(1,2)     | 1    | 1    | 3    |
| U(1,3)     | 1    | 1    | 1    |
| U(2,3)     | 1    | 1    | 1    |

`U(2,3)`에서 두 노드의 루트노드가 서로 같음을 통해 사이클이 있는 걸 알게된다.

In [4]:
# 노드의 개수와 간선(Union 연산)의 개수 입력 받기
v, e = map(int, input().split())
parent = [0] * (v + 1) # 부모 테이블 초기화하기

# 부모 테이블 상에서, 부모를 자기 자신으로 초기화
for i in range(1, v+1):
    parent[i] = i
    
cycle = False # 사이클 발생 여부
    
# Union 연산을 각각 수행
for i in range(e):
    a, b = map(int, input().split())
    if find_parent(parent, a) == find_parent(parent, b):
        cycle = True
    else:
        union_parent(parent, a, b)

if cycle:
    print('사이클 있다.')
else:
    print('사이클 없다.')

3 3
1 2
2 3
1 3
사이클 있다.


### 거짓말
https://www.acmicpc.net/problem/1043

In [44]:
def find_parent(parent, a):
    if parent[a] != a:
        parent[a] = find_parent(parent, parent[a])
    return parent[a]

def union(parent, a, b):
    root_a = find_parent(parent, a)
    root_b = find_parent(parent, b)
    if root_a > root_b:
        parent[root_a] = root_b
    else:
        parent[root_b] = root_a

n_person, n_party = map(int, input().split())
n_knower, *knowers = map(int, input().split())

parent = [None] + [i for i in range(1, n_person+1)]

for a, b in zip(knowers[:n_knower-1], knowers[1:n_knower]):
    union(parent, a, b)

party_list = []
for _ in range(n_party):
    n, *people = map(int, input().split())
    party_list.append(people)
    if n == 1:
        continue
    for a, b in zip(people[:n-1], people[1:]):
        union(parent, a, b)

ct = 0
if len(knowers) == 0:
    print(n_party)
else:
    knower = knowers[0]
    for party in party_list:
        in_knower = any([find_parent(parent, p) == find_parent(parent, knower) for p in party])
        if not in_knower:
            ct += 1
    print(ct)

4 5
1 1
1 1
1 2
1 3
1 4
2 4 1
2


# 최소신장트리, 위상정렬은 나중에

# 위상정렬 for DAG

1. 순방향방법 (with bfs)

   > DAG에서 진입차수가 0인 v부터 시작하여 제거하면서 진입차수가 0인 v들을 계속 뱉는 방법

2. 역방향방법 (with dfs)

   > DAG에서 진출 차수가 0인 v부터 시작하여 제거하면서 진출차수가 0인 v들을 계속 뱉는 방법

#### 줄 세우기

https://www.acmicpc.net/problem/2252

계획

> 1. DAG를 구현한다.
> 2. 위상정렬을 시행한다.
> > 1. 차수가 0인 vertex들은 queue에 넣는다. 
> > 2. 터트리고 차수를 1씩 낮춘다. and repeat 1

In [78]:
from collections import deque

N, M = list(map(int, input().split()))

# DAG 구현
degrees = [0 for i in range(N+1)]
adj_list = [[] for i in range(N+1)]

for i in range(M):
    from_, to_ = list(map(int, input().split()))
    adj_list[from_].append(to_)
    degrees[to_]+=1

# 위상정렬 with bfs
q = deque()
for i in range(1, N+1):
    if degrees[i]==0:
        q.append(i)
        
results = []

while len(q)!=0:
    from_ = q.popleft()
    to_ = adj_list[from_]
    results.append(from_)
    
    for to in to_:
        degrees[to] -= 1
        if degrees[to]==0:
            q.append(to)

for i in results:
    print(i, end=' ')

4 2
4 2
1 3
1 4 3 2 

#### ACM Craft
https://www.acmicpc.net/problem/1005

계획

>1. DAG 구현
>2. 위상 정렬
>3. 각 vertex마다 cost 쌓아가기

In [5]:
from collections import deque

t = int(input())

for xxxx in range(t):
    N, M = list(map(int, input().split()))

    # DAG 구현
    degrees = [0 for i in range(N+1)]
    adj_list = [[] for i in range(N+1)]
    costs = [0] + list(map(int, input().split()))
    dists = costs.copy()

    for i in range(M):
        from_, to_ = list(map(int, input().split()))
        adj_list[from_].append(to_)
        degrees[to_]+=1

    dest = int(input())    

    # 위상정렬 with bfs
    q = deque()
    for i in range(1, N+1):
        if degrees[i]==0:
            q.append(i)

    # results = []

    while len(q)!=0:
        f = q.popleft()
        t = adj_list[f]
        # results.append(f)

        for to in t:
            degrees[to] -= 1
            dists[to] = max(dists[to], dists[f]+costs[to])
            if degrees[to]==0:
                q.append(to)

    print(dists[dest])

2
4 4
10 1 100 10
1 2
1 3
2 4
3 4
4
120
8 8
10 20 1 5 8 7 1 43
1 2
1 3
2 4
2 5
3 6
5 7
6 7
7 8
7
39


#### 작업
https://www.acmicpc.net/problem/2056

In [22]:
from collections import deque

N = int(input())

# DAG 구현
degrees = [0 for i in range(N+1)]
adj_list = [[] for i in range(N+1)]
costs = [0]*(N+1)

for i in range(1, N+1):
    cost, degree, *inv_adj = list(map(int, input().split()))
    costs[i] = cost
    degrees[i] = degree
    for inv in inv_adj:
        adj_list[inv].append(i)
dists = costs.copy()


# 위상정렬 with bfs
q = deque()
for i in range(1, N+1):
    if degrees[i]==0:
        q.append(i)

while len(q)!=0:
    f = q.popleft()
    t = adj_list[f]
    for to in t:
        degrees[to] -= 1
        dists[to] = max(dists[to], dists[f]+costs[to])
        if degrees[to]==0:
            q.append(to)
print(max(dists))

7
5 0
1 1 1
3 1 2
6 1 1
1 2 2 4
8 2 2 4
4 3 3 5 6


#### 최소스패닝트리
https://www.acmicpc.net/problem/1197

계획

1. edge를 weights가 낮은 순서대로 가장 작은 순서대로 (a,b) 나열한다.

2. 각 (a,b)마다 find를 하여서 서로 부모노드가 다르면 union을한다.

In [1]:
N, M = map(int, input().split())

weights = []
for _ in range(M):
    weights.append(tuple(map(int, input().split())))

group = [i for i in range(N+1)] # 우선은 노드 자기 자신이 그륩 representative

def find(u):
    if u!=group[u]:
        group[u] = find(group[u])
    return group[u]

def union(u,v):
    root1 = find(u)
    root2 = find(v)
    group[root2] = root1

weights.sort(key = lambda x: x[2])

mst = []

# 최소 스패닝 트리는 N-1개의 edge를 정의하는 것이다.
i = 0
while len(mst)!=N-1:
    u, v = weights[i][0], weights[i][1]
    if find(u)==find(v):
        i += 1
        continue
    else:
        union(u,v)
        mst.append((weights[i]))
        i +=1

print(sum([i[2] for i in mst]))

3 3
1 2 1
2 3 2
1 3 3
3


# 4. 다익스트라 (최단경로)

1. 출발 노드를 설정
2. 출발 노드를 기준으로 각 노드의 최소 비용을 저장
3. 인접노드를 우선순위 큐에 넣는다
4. 해당 노드를 거쳐서 갈 수 있는 노드를 고려한다.
5. repeat 3

![](./fig/dij/dij.gif)


| 노드1 | 노드2 | 노드3 | 노드4 | 노드5 | 노드6   |
| ----- | ----- | ----- | ----- | ----- | ------- |
| (0)     | 2     | 5     | 1     | 무한  | 무한    |
| 0     | 2     | 5=>4  | (1)     | 2     | 무한    |
| 0     | (2)     | 4     | 1     | 2     | 무한    |
| 0     | 2     | 4=>3  | 1     | (2)     | 무한=>4 |
| 0     | 2     | (3)     | 1     | 2     | 4       |
| 0     | 2     | 3     | 1     | 2     | (4)       |

In [8]:
import heapq

INF = int(1e9)

n, m = map(int, input().split())
start = int(input())
graph = [[] for i in range(n+1)]
distance = [INF] * (n+1)

for _ in range(m):
    a, b, c = map(int, input().split())
    graph[a].append((b, c)) # a노드에서 b노드로 가는 비용이 c다.
    graph[b].append((a, c))

6 10
1
1 2 2
1 3 5
1 4 1
2 3 3
2 4 2
3 4 3
3 5 1
3 6 5
4 5 1
5 6 2


In [9]:
def dijkstra(start):
    q = []
    heapq.heappush(q, (0, start)) # (cost, node)
    distance[start] = 0
    # visted[start] = True
    
    while q: # 큐가 비어있지 않다면
        # 가장 최단 거리가 짧은 노드 꺼내기
        dist, now = heapq.heappop(q)
        # 현재 노드가 이미 처리된 적이 있다면 무시
        if distance[now] < dist:
            continue
            
        # 3. 인접노드들 우선순위 큐에 넣는다
        for (b, c) in graph[now]:
            cost = dist + c
            # 4. 해당 노드를 거쳐서 갈 수 있는 노드를 고려한다.
            if cost < distance[b]:
                distance[b] = cost
                heapq.heappush(q, (cost, b))

In [12]:
distance

[1000000000, 0, 2, 3, 1, 2, 4]

### 특정한 최단 경로
https://www.acmicpc.net/problem/1504

In [14]:
import heapq

def dijkstra(start, distance):
    q = []
    heapq.heappush(q, (0, start)) # (cost, node)
    distance[start] = 0
    
    while q: # 큐가 비어있지 않다면
        # 가장 최단 거리가 짧은 노드 꺼내기
        dist, now = heapq.heappop(q)
        # 현재 노드가 이미 처리된 적이 있다면 무시
        if distance[now] < dist:
            continue
            
        # 3. 인접노드들 우선순위 큐에 넣는다
        for (b, c) in graph[now]:
            cost = dist + c
            # 4. 해당 노드를 거쳐서 갈 수 있는 노드를 고려한다.
            if cost < distance[b]:
                distance[b] = cost
                heapq.heappush(q, (cost, b))

INFTY = 1e5

n_node, n_edge = map(int, input().split())


graph = [[] for _ in range(n_node+1)]
distance1 = [INFTY for _ in range(n_node+1)]
distance2 = [INFTY for _ in range(n_node+1)]
distance3 = [INFTY for _ in range(n_node+1)]

for _ in range(n_edge):
    a, b, c = map(int, input().split())
    graph[a].append((b, c))
    graph[b].append((a, c))

mid1, mid2 = map(int, input().split())

start = 1
end = n_node

dijkstra(start, distance1)
dijkstra(mid1, distance2)
dijkstra(mid2, distance3)

cost = min(distance1[mid1] + distance2[mid2] + distance3[end], distance1[mid2] + distance3[mid1] + distance2[end])

if cost < INFTY:
    print(cost)
else:
    print(-1)

4 6
1 2 3
2 3 3
3 4 1
1 3 5
2 4 5
1 4 4


### 파티

https://www.acmicpc.net/problem/1238

다익스트라 1000번

In [ ]:
4 8 2
1 2 4
1 3 2
1 4 7
2 1 1
2 3 5
3 1 2
3 4 4
4 2 3

In [43]:
N, M, X = map(int, input().split())
graph = [[] for _ in range(N+1)]
for _ in range(M):
    start, dest, dist = map(int, input().split())
    graph[start].append((dest, dist))

import heapq
INFTY = 1e9
def dijkstra(start):
    distance = [INFTY] * (N+1)
    q = []
    heapq.heappush(q, (start, 0))
    distance[start] = 0

    while q:
        now, dist = heapq.heappop(q)
        if distance[now] < dist:
            continue
        
        for (c_node, c_dist) in graph[now]:
            cost = dist + c_dist
            if cost < distance[c_node]:
                distance[c_node] = cost
                heapq.heappush(q, (c_node, cost))
    
    return distance

temp = [dijkstra(i) for i in range(N+1)]
print(max([temp[i][X] + temp[X][i] for i in range(1, N+1)]))

4 8 2
1 2 4
1 3 2
1 4 7
2 1 1
2 3 5
3 1 2
3 4 4
4 2 3
10


### 최단경로
https://www.acmicpc.net/problem/1753

In [23]:
import heapq

n_node, n_edge = map(int, input().split())

start = int(input())

graph = [[] for _ in range(n_node+1)]

for _ in range(n_edge):
    a, b, c = map(int, input().split())
    graph[a].append((b, c))
###
# ALGO
###
INFTY = 1e10
cost = [INFTY] * (n_node + 1)
q = []
heapq.heappush(q, (0, start))
cost[start] = 0

while q:
    cost_candidate, node = heapq.heappop(q)
    if cost[node] < cost_candidate:
        continue
    for b, c in graph[node]:
        cost_b = cost_candidate + c
        if cost[b] > cost_b:
            heapq.heappush(q, (cost_b, b))
            cost[b] = cost_b
###
# ANS
###
for c in cost[1:]:
    if c == INFTY:
        print('INF')
    else:
        print(c)

5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6
0
2
3
7
INF


In [ ]:
5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6